In [1]:
%run helper/setup_notebook.ipynb

Successfully connected to sql_lab database.


**LAG Function Syntax:**

The LAG function allows you to retrieve a value from a previous row in a result set based on a specified offset. Here is the syntax for using the LAG function:

```sql
LAG(expression [, offset [, default]]) OVER ([query_partition_clause] order_by_clause)
```

- **expression (mandatory):** This represents the column or value that you want the LAG function to evaluate and return for the previous row.

- **offset (optional):** You can specify the number of rows "backward" in the result set to look at. If you omit this parameter, the default offset is 1, meaning the LAG function will retrieve the value from the immediate previous row.

- **default (optional):** If the offset is outside the bounds of the result set, you can specify a default value to be returned. The default value is usually set to NULL if not explicitly provided.

- **query_partition_clause (optional):** This clause allows you to divide the analysis into partitions, similar to other analytic functions. It creates groups within which the LAG function operates independently.

- **order_by_clause (mandatory):** Use the order_by_clause to determine the order of the records in the result set. This determines which row is considered "previous" for the LAG function.

In [2]:
%%sql 

SELECT * FROM stock_price;


stock_name,price_date,closing_price
Apple,2023-01-01,155.00
Apple,2023-01-02,160.00
Apple,2023-01-03,143.00
Apple,2023-01-04,169.00
Apple,2023-01-05,162.00
Microsoft,2023-01-01,205.00
Microsoft,2023-01-02,211.00
Microsoft,2023-01-03,202.00
Microsoft,2023-01-04,226.00
Microsoft,2023-01-05,214.00


In [3]:
%%sql 
-- Notice the incorrect previous_day_close values without partitioning
SELECT 
    stock_name,
    price_date,
    closing_price,
    LAG(closing_price) OVER (ORDER BY price_date) AS previous_day_close
FROM stock_price
ORDER BY stock_name;

stock_name,price_date,closing_price,previous_day_close
Apple,2023-01-01,155.00,None
Apple,2023-01-02,160.00,205.00
Apple,2023-01-03,143.00,211.00
Apple,2023-01-04,169.00,202.00
Apple,2023-01-05,162.00,226.00
Microsoft,2023-01-01,205.00,155.00
Microsoft,2023-01-02,211.00,160.00
Microsoft,2023-01-03,202.00,143.00
Microsoft,2023-01-04,226.00,169.00
Microsoft,2023-01-05,214.00,162.00


In [4]:
%%sql 

SELECT 
    stock_name,
    price_date,
    closing_price,
    LAG(closing_price) OVER(PARTITION BY stock_name ORDER BY price_date) AS previous_day_close,
    closing_price - (LAG(closing_price) OVER (PARTITION BY stock_name ORDER BY price_date)) AS day_variance
FROM stock_price
ORDER BY stock_name;

stock_name,price_date,closing_price,previous_day_close,day_variance
Apple,2023-01-01,155.00,None,None
Apple,2023-01-02,160.00,155.00,5.00
Apple,2023-01-03,143.00,160.00,-17.00
Apple,2023-01-04,169.00,143.00,26.00
Apple,2023-01-05,162.00,169.00,-7.00
Microsoft,2023-01-01,205.00,None,None
Microsoft,2023-01-02,211.00,205.00,6.00
Microsoft,2023-01-03,202.00,211.00,-9.00
Microsoft,2023-01-04,226.00,202.00,24.00
Microsoft,2023-01-05,214.00,226.00,-12.00


In [5]:
%%sql 
-- with default value
SELECT 
    stock_name,
    price_date,
    closing_price,
    LAG(closing_price, 1, 'No prev day info') OVER(PARTITION BY stock_name ORDER BY price_date) AS previous_day_close,
    closing_price - (LAG(closing_price) OVER (PARTITION BY stock_name ORDER BY price_date)) AS day_variance
FROM stock_price
ORDER BY stock_name;

stock_name,price_date,closing_price,previous_day_close,day_variance
Apple,2023-01-01,155.00,No prev day info,None
Apple,2023-01-02,160.00,155.00,5.00
Apple,2023-01-03,143.00,160.00,-17.00
Apple,2023-01-04,169.00,143.00,26.00
Apple,2023-01-05,162.00,169.00,-7.00
Microsoft,2023-01-01,205.00,No prev day info,None
Microsoft,2023-01-02,211.00,205.00,6.00
Microsoft,2023-01-03,202.00,211.00,-9.00
Microsoft,2023-01-04,226.00,202.00,24.00
Microsoft,2023-01-05,214.00,226.00,-12.00


#### Find the biggest gain day for Apple.

In [12]:
%%sql 

-- suboptimal option

SELECT
    DATE_FORMAT(price_date, '%b-%d') AS price_date,
    closing_price - (LAG(closing_price) OVER (PARTITION BY stock_name ORDER BY price_date)) AS day_variance
FROM stock_price
WHERE stock_name = 'Apple'
ORDER BY day_variance DESC
LIMIT 1;

price_date,day_variance
Jan-04,26.00


In [20]:
%%sql 

-- better and more flexible option using CTE

WITH growth_CTE AS (
    SELECT
        DATE_FORMAT(price_date, '%b-%d') AS price_date,
        closing_price - (LAG(closing_price) OVER (PARTITION BY stock_name ORDER BY price_date)) AS day_variance
    FROM stock_price
    WHERE stock_name = 'Apple'
)
SELECT 
    price_date,
    day_variance
FROM growth_CTE
WHERE day_variance = ( SELECT MAX(day_variance) FROM growth_CTE)

price_date,day_variance
Jan-04,26.00
